In [124]:
import numpy as np
import math
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
import timeit

In [116]:
def read_input(filename):
    input_data=[];
    f=open(filename);
    for line in f:
        input_data.append(line.split());
    return input_data;

def create_feature_matrix(input_data):
    x_list=[a[0:len(input_data[0])-1] for a in input_data];
    x=np.matrix(x_list);
    #x=np.transpose(x)
    return x.astype(np.float);

def create_y_matrix(filename):
    y_list=[a[len(input_data[0])-1] for a in input_data];
    y=np.matrix(y_list);
    y=np.transpose(y)
    return y.astype(np.float);

def poly_model_multiple(training_data_x, training_data_y, poly_degree):
    poly = PolynomialFeatures(poly_degree);
    Z=poly.fit_transform(training_data_x);
    theta=np.dot(np.linalg.pinv(Z), training_data_y);
    return theta;

def predict(testing_data_x, theta, degree):
    poly = PolynomialFeatures(degree);
    Z=poly.fit_transform(testing_data_x);
    predicted_y=np.dot(Z, theta);
    return predicted_y;

def relative_mean_square(predicted_y, training_data_y):
    difference_y_num=np.empty([predicted_y.shape[0], predicted_y.shape[1]]);
    difference_y_deno=np.empty([predicted_y.shape[0], predicted_y.shape[1]]);
    
    for i in range(predicted_y.shape[0]):
        for j in range(predicted_y.shape[1]):
            difference_y_num[i][j]=1.*predicted_y[i][j]-training_data_y[i][j];
            difference_y_num[i][j]=1.*(difference_y_num[i][j]*difference_y_num[i][j])
            #difference_y_deno[i][j]=1.*training_data_y[i][j]-np.average(training_data_y);
            #difference_y_deno[i][j]=1.*(difference_y_deno[i][j]*difference_y_deno[i][j])
    #print difference_y_num
    difference_y_num=1.*difference_y_num.sum()
    #print difference_y_deno
    #difference_y_deno=1.*difference_y_deno.sum()
    #RMS=1.*(difference_y_num/difference_y_deno)/len(predicted_y);
    RMS=1.*(difference_y_num)/len(predicted_y);
    
    #print RMS
    return RMS;
    
def do_cross_validation(data_x, data_y,degree,  n_folds=10):
    cv = KFold(len(data_y), n_folds)
    error_mean_custom = []
    error_mean_python = []
    
    i=1;
    for train_idx, test_idx in cv:
        theta=poly_model_multiple(data_x[train_idx], data_y[train_idx], degree);
        #print "(Custom Model) For fold: %d theta = " %i
        #print theta
        poly=PolynomialFeatures(degree);
        X_train=poly.fit_transform(data_x[train_idx])
        clf=LinearRegression();
        clf.fit(X_train, data_y[train_idx])
        X_test=poly.fit_transform(data_x[test_idx])
        predicted_y_python=clf.predict(X_test)

        #print "(Python Model) For fold: %d theta =" %i
        #print clf.intercept_, clf.coef_
        predicted_y_custom=predict(data_x[test_idx], theta, degree)
        RMS_custom=relative_mean_square(predicted_y_custom, data_y[test_idx]);
        RMS_python=relative_mean_square(predicted_y_python, data_y[test_idx]);
        print "(Custom Model) For fold: %d RMS = %.15f" %(i, RMS_custom)
        print "(Python Model) For fold: %d RMS = %.15f" %(i, RMS_python)
        error_mean_custom.append(RMS_custom);
        error_mean_python.append(RMS_python);
        i=i+1;
    avg_custom=np.mean(error_mean_custom);
    avg_python=np.mean(error_mean_python);
    #print "aaaa"
    #print (avg_custom)
    return avg_custom, avg_python;

def plot_graph(data_x, data_y, predicted_y, predicted_y_python):
    plt.plot( data_y,'+');
    plt.plot( predicted_y,'o');
    plt.plot( predicted_y_python,'.', color='red');
    plt.show();

"""input_data=read_input("mvar-set1.dat")
data_x=create_feature_matrix(input_data)
data_y=create_y_matrix(input_data)
degree=2
poly=PolynomialFeatures(degree);
avg_custom, avg_python=do_cross_validation(data_x, data_y, degree, 10)
print "(Custom & Python Model) Average RMS: %f , %f" %(avg_custom, avg_python)
theta=poly_model_multiple(data_x, data_y, degree)
predicted_y=predict(data_x, theta, degree);
#plot_graph( data_x, data_y, predicted_y);
#X=poly.fit_transform(data_x)
#clf=LinearRegression();
#clf.fit(X, data_y);
#predicted_y_python=clf.predict(X);
#plot_graph( data_x, data_y, predicted_y,predicted_y_python);

input_data=read_input("mvar-set2.dat")
data_x=create_feature_matrix(input_data)
data_y=create_y_matrix(input_data)
avg_custom, avg_python=do_cross_validation(data_x, data_y,degree, 10)
print "(Custom & Python Model) Average RMS: %f , %f" %(avg_custom, avg_python)
theta=poly_model_multiple(data_x, data_y, degree)
predicted_y=predict(data_x, theta, degree);
#plot_graph( data_x, data_y, predicted_y);
#X=poly.fit_transform(data_x)
#clf=LinearRegression();
#clf.fit(X, data_y);
#predicted_y_python=clf.predict(X);
#plot_graph( data_x, data_y, predicted_y,predicted_y_python);
"""

degree=4
input_data=read_input("mvar-set1.dat")
data_x=create_feature_matrix(input_data)
data_y=create_y_matrix(input_data)
avg_custom, avg_python=do_cross_validation(data_x, data_y,degree, 10)
print "(Custom & Python Model) Average RMS: %.15f, %.15f" %(avg_custom, avg_python)
theta=poly_model_multiple(data_x, data_y, degree)
predicted_y=predict(data_x, theta, degree);




(Custom Model) For fold: 1 RMS = 0.265650340716356
(Python Model) For fold: 1 RMS = 0.265650340716356
(Custom Model) For fold: 2 RMS = 0.262614536990708
(Python Model) For fold: 2 RMS = 0.262614536990708
(Custom Model) For fold: 3 RMS = 0.236250757575800
(Python Model) For fold: 3 RMS = 0.236250757575800
(Custom Model) For fold: 4 RMS = 0.252117799395808
(Python Model) For fold: 4 RMS = 0.252117799395808
(Custom Model) For fold: 5 RMS = 0.272634237262345
(Python Model) For fold: 5 RMS = 0.272634237262345
(Custom Model) For fold: 6 RMS = 0.285770428701594
(Python Model) For fold: 6 RMS = 0.285770428701594
(Custom Model) For fold: 7 RMS = 0.265685748863074
(Python Model) For fold: 7 RMS = 0.265685748863074
(Custom Model) For fold: 8 RMS = 0.214167715431208
(Python Model) For fold: 8 RMS = 0.214167715431208
(Custom Model) For fold: 9 RMS = 0.282958360190345
(Python Model) For fold: 9 RMS = 0.282958360190344
(Custom Model) For fold: 10 RMS = 0.265204845915394
(Python Model) For fold: 10 RM

In [105]:
degree=4
input_data=read_input("mvar-set2.dat")
data_x=create_feature_matrix(input_data)
data_y=create_y_matrix(input_data)
avg_custom, avg_python=do_cross_validation(data_x, data_y,degree, 10)
print "(Custom & Python Model) Average RMS: %.15f, %.15f" %(avg_custom, avg_python)
theta=poly_model_multiple(data_x, data_y, degree)
predicted_y=predict(data_x, theta, degree);

(Custom Model) For fold: 1 RMS = 0.010612912738546
(Python Model) For fold: 1 RMS = 0.010612912738546
(Custom Model) For fold: 2 RMS = 0.010803370283764
(Python Model) For fold: 2 RMS = 0.010803370283764
(Custom Model) For fold: 3 RMS = 0.010329785169964
(Python Model) For fold: 3 RMS = 0.010329785169964
(Custom Model) For fold: 4 RMS = 0.011833258555848
(Python Model) For fold: 4 RMS = 0.011833258555848
(Custom Model) For fold: 5 RMS = 0.009744908486242
(Python Model) For fold: 5 RMS = 0.009744908486242
(Custom Model) For fold: 6 RMS = 0.010208544234663
(Python Model) For fold: 6 RMS = 0.010208544234663
(Custom Model) For fold: 7 RMS = 0.009976716905275
(Python Model) For fold: 7 RMS = 0.009976716905275
(Custom Model) For fold: 8 RMS = 0.010338641151329
(Python Model) For fold: 8 RMS = 0.010338641151329
(Custom Model) For fold: 9 RMS = 0.010608544576755
(Python Model) For fold: 9 RMS = 0.010608544576755
(Custom Model) For fold: 10 RMS = 0.009935997101544
(Python Model) For fold: 10 RM

In [125]:
start_time=timeit.default_timer()
degree=3
input_data=read_input("mvar-set3.dat")
data_x=create_feature_matrix(input_data)
data_y=create_y_matrix(input_data)
avg_custom, avg_python=do_cross_validation(data_x, data_y,degree, 10)
print "(Custom & Python Model) Average RMS: %.15f, %.15f" %(avg_custom, avg_python)
theta=poly_model_multiple(data_x, data_y, degree)
predicted_y=predict(data_x, theta, degree);
print "The total time taken by Poly Model is: %.15f" %(timeit.default_timer() - start_time)

(Custom Model) For fold: 1 RMS = 0.246599145246067
(Python Model) For fold: 1 RMS = 0.246599145246067
(Custom Model) For fold: 2 RMS = 0.252285576991936
(Python Model) For fold: 2 RMS = 0.252285576991936
(Custom Model) For fold: 3 RMS = 0.252617719932084
(Python Model) For fold: 3 RMS = 0.252617719932084
(Custom Model) For fold: 4 RMS = 0.251760630139349
(Python Model) For fold: 4 RMS = 0.251760630139349
(Custom Model) For fold: 5 RMS = 0.254205904286961
(Python Model) For fold: 5 RMS = 0.254205904286961
(Custom Model) For fold: 6 RMS = 0.251485055155519
(Python Model) For fold: 6 RMS = 0.251485055155519
(Custom Model) For fold: 7 RMS = 0.250481999190894
(Python Model) For fold: 7 RMS = 0.250481999190894
(Custom Model) For fold: 8 RMS = 0.248422024632932
(Python Model) For fold: 8 RMS = 0.248422024632932
(Custom Model) For fold: 9 RMS = 0.249205082080232
(Python Model) For fold: 9 RMS = 0.249205082080232
(Custom Model) For fold: 10 RMS = 0.252373476169047
(Python Model) For fold: 10 RM

In [99]:
degree=2
input_data=read_input("mvar-set4.dat")
data_x=create_feature_matrix(input_data)
data_y=create_y_matrix(input_data)
avg_custom, avg_python=do_cross_validation(data_x, data_y,degree, 10)
print "(Custom & Python Model) Average RMS: %.15f, %.15f" %(avg_custom, avg_python)
theta=poly_model_multiple(data_x, data_y, degree)
predicted_y=predict(data_x, theta, degree);

(Custom Model) For fold: 1 RMS = 0.246552459096136
(Python Model) For fold: 1 RMS = 0.246552459096136
(Custom Model) For fold: 2 RMS = 0.252056867810695
(Python Model) For fold: 2 RMS = 0.252056867810695
(Custom Model) For fold: 3 RMS = 0.252500120490041
(Python Model) For fold: 3 RMS = 0.252500120490041
(Custom Model) For fold: 4 RMS = 0.251629351011797
(Python Model) For fold: 4 RMS = 0.251629351011797
(Custom Model) For fold: 5 RMS = 0.254145574468101
(Python Model) For fold: 5 RMS = 0.254145574468100
(Custom Model) For fold: 6 RMS = 0.251497197248847
(Python Model) For fold: 6 RMS = 0.251497197248847
(Custom Model) For fold: 7 RMS = 0.250425122212788
(Python Model) For fold: 7 RMS = 0.250425122212788
(Custom Model) For fold: 8 RMS = 0.248350093539740
(Python Model) For fold: 8 RMS = 0.248350093539740
(Custom Model) For fold: 9 RMS = 0.249150989613195
(Python Model) For fold: 9 RMS = 0.249150989613195
(Custom Model) For fold: 10 RMS = 0.252055571708536
(Python Model) For fold: 10 RM